In [8]:
def getUrl(url):
    response = requests.get(url)
    pdfUrl = []
    count = 0
    bs = BeautifulSoup(response.text,"html.parser")
    pdfs = bs.find_all(href=re.compile('/content/.*html'))
    names = []
    for tag in pdfs:
        if count < 20: 
            pdfUrl.append("https://openaccess.thecvf.com"+tag.get("href"))
            count = count+1
            names.append(re.split('[./]', tag.get("href"))[-2][:-16].replace("_"," "))
        else: break
    return names,pdfUrl


def getAbstract(paperUrls):
    abstract = []
    for url in paperUrls:
        response = requests.get(url)
        bs = BeautifulSoup(response.text,"html.parser")
        abstract.append(bs.find(id="abstract").text.strip())
    return abstract

def translate_api(input_text):
    # Set your own appid/appkey.需要自己申请，处于隐私安全考虑，我就不放上来了，可以当场演示
    appid = ''
    appkey = ''
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()
    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang = 'zh'
    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    
    query = input_text
    # Generate salt and sign
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)
    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()
    return result['trans_result'][0]['dst']

def write2txt(path,title,contents):
    with open(path,"w") as f:
        for ti,con in zip(title,contents):
            f.write(ti+"\n")
            f.write(con+"\n")
            f.write("\n")


In [2]:
import requests
from bs4 import BeautifulSoup
import re
import random
from hashlib import md5

In [3]:
web = "https://openaccess.thecvf.com/ICCV2021?day=2021-10-12"
name,pdfurls = getUrl(web)
abstracts = getAbstract(pdfurls)
ch_abstracts = []

In [9]:
for ab in abstracts:
    ch_abstracts.append(translate_api(ab))
write2txt("./myresult2021.txt",name,ch_abstracts)
